<font size = 6 >RT_02 </font>

<font size = 4 color = 'Red'>Student : </font>
<font size = 4 >Hafiz Muizz Ahmed Sethi</font>

<font size = 4 color = 'Red'>Student ID: </font>
<font size = 4> S5428151</font>

# Notebook description
<font size = 4 > This Notebook acts as a user interface for controlling a two-wheeled mobile robot within a ROS environment. It provides a graphical representation of the robot's state and behavior. </font> 


<font size = 4 > Node 'A' incorporates an action client that enables the user to specify a target goal or cancel it. Additionally, the node publishes velocity information using a custom message format that includes values for position (x, y) and velocity (x_vel, y_vel), derived from the data published in the "/odom" topic.</font>

<font size = 4 > To replace Node 'A', we intend to develop a Jupyter Notebook. This notebook will utilize widgets to provide the user with the following information: </font>

<font size = 4 >  The current position of the robot </font>

<font size = 4 >  A log of set goals and cancelled goals</font>

<font size = 4 >  The distance to the nearest obstacle </font>

<font size = 5 >1.To Start we have to add Necessary extensions </font>

In [2]:
from ipywidgets import VBox,HBox,Layout
import ipywidgets as widgets
import jupyros as jr
import rospy
from IPython.display import display, clear_output
import time
import numpy as np
from nav_msgs.msg import Odometry
from geometry_msgs.msg import  Twist
from robot_r.srv import my_service, my_serviceResponse
from sensor_msgs.msg import LaserScan 
from assignment_2_2022.msg import PlanningAction, PlanningGoal
from assignment_2_2022.msg import  PlanningActionResult, PlanningActionGoal,PlanningActionFeedback
from geometry_msgs.msg import PoseStamped
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import actionlib
from math import pi


### Node initialization and Declaration of pusblisher.

In [ ]:

rospy.init_node('user_interface', anonymous=True)

publisher = rospy.Publisher('cmd_vel',Twist, queue_size=10)


client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)




### Buttons for send and cancel Goal 

In [3]:
x_goal=widgets.BoundedFloatText(value=0.0,min=-10.0,max=10.0, step=0.1,layout=widgets.Layout(width='200px'), description='X_position:',disabled=False)
y_goal=widgets.BoundedFloatText(value=0.0,min=-10.0,max=10.0, step=0.1,layout=widgets.Layout(width='200px'), description='Y_position:',disabled=False)

button1_goal = widgets.Button(description="Send Goal", button_style='' 'info' '')
button2_cancel=widgets.Button(description="Cancel target",button_style='' 'danger' '')

#display(button1_goal)
#display(button2_cancel)
#display(x_goal)
#display(y_goal)


### Buttons Layout

In [4]:
sendig_goal_box = widgets.VBox([
    
    widgets.HBox([x_goal]),
    widgets.HBox([y_goal]),
    widgets.HBox([button1_goal, button2_cancel])
])

# Display the layout
#display(sendig_goal_box)


### Goal Statue display

In [5]:
disp = widgets.Textarea(value='', description='Goal Status')
#disp

### Callback function for sending goal Buttons

In [6]:
def button1_Goal_callback(b):
    
    client.wait_for_server() 
    
    goal = PoseStamped()    
    goal.pose.position.x = x_goal.value
    goal.pose.position.y = y_goal.value
    #setting the goal, using the desired x,y of the user.
    goal = PlanningGoal(goal) 
    #sending the goal       
    client.send_goal(goal)
    status_goal = True
    goal_text = 'Your goal is set x= ({}, {})'.format(x_goal.value, y_goal.value)
    disp.value = goal_text       
            
            
def button2_cancel_callback(b):
    client.cancel_goal()
    calcel_text='Goal Cancelled'
    disp.value=calcel_text
    
button1_goal.on_click(button1_Goal_callback)
button2_cancel.on_click(button2_cancel_callback)
#display(out)

### Buttons for moving The Robot in Different Directions

In [7]:
button_backward = widgets.Button(description="BACKWARD", button_style='' 'info' '')
button_forward=widgets.Button(description="FORWARD",button_style='' 'info' '')
button_left = widgets.Button(description="LEFT", button_style='' 'info' '')
button_right=widgets.Button(description="RIGHT",button_style='' 'info' '')

#display(button_left)
#display(button_right)
#display(button_forward)
#display(button_backward)

### Buttons Layout

In [8]:
direction_buttons_box = widgets.VBox([
    widgets.HBox([widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''), button_forward, widgets.Label('')]),
    widgets.HBox([button_left, widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''), button_right]),
    widgets.HBox([widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''), button_backward, widgets.Label('')]),
])
#display(direction_buttons_box)

### Callback Functions for Moving Robot in different Directions Buttons

In [ ]:
vel=Twist()

def button_Backward_Callback(b):
    vel.linear.x=-1
    publisher.publish(vel)
    time.sleep(1)
    vel.linear.x=0
    publisher.publish(vel)
    
    
def button_Forward_Callback(b):
    vel.linear.x=1
    publisher.publish(vel)
    time.sleep(1)
    vel.linear.x=0
    publisher.publish(vel)
    
    
def button_Left_Callback(b):
    vel.angular.z=1
    publisher.publish(vel)
    time.sleep(1)
    vel.angular.z=0
    publisher.publish(vel)
    

def button_Right_Callback(b):
    vel.angular.z=-1
    publisher.publish(vel)
    time.sleep(1)
    vel.angular.z=0
    publisher.publish(vel)
    

button_left.on_click(button_Left_Callback)
button_right.on_click(button_Right_Callback)
button_forward.on_click(button_Forward_Callback)
button_backward.on_click(button_Backward_Callback)


### Collecting Previous Widgets in one box

In [ ]:
vbox=widgets.VBox([sendig_goal_box,disp])
hbox = widgets.HBox([vbox,widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''),widgets.Label(''), direction_buttons_box])



###  Plot of Robot Position and Plot for Reached and Calcelled Goals

In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots(2, 1, figsize=(8, 10))
        self.robot_ln, = self.ax[1].plot([], [], 'o', label='Robot Position')
        self.target_ln, = self.ax[1].plot([], [], 'x', label='Target Position')

        goal = PoseStamped()
        self.x_robot = []
        self.y_robot = [] 
        self.x_target = []
        self.y_target = []

        self.x = ["Cancelled", "Reached"]
        self.y = [0, 0]
        self.reached_bar = self.ax[0].bar(self.x, self.y, color=['red', 'green'], edgecolor='black', linewidth=1, width=0.5)

    def plot_init(self):
        self.ax[1].set_xlim(-10, 10)
        self.ax[1].set_ylim(-10, 10)
        self.ax[1].set_title('Robot and Target Position')
        self.ax[1].set_xlabel('x_pos')
        self.ax[1].set_ylabel('y_pos')
        self.ax[1].legend()

        self.ax[0].set_xlim(0, 2)
        self.ax[0].set_ylim(0, 5)
        self.ax[0].set_title('Target Status Count')
        self.ax[0].set_xlabel('Status')
        self.ax[0].set_ylabel('')

        self.fig.subplots_adjust(hspace=0.5)

        return self.robot_ln, self.target_ln, self.reached_bar

    def odom_callback(self, msg):
        self.y_robot.append(msg.pose.pose.position.y)
        self.x_robot.append(msg.pose.pose.position.x)
    
    def TargetPoseCallback(self, goal: PlanningActionGoal):
        self.x_target = goal.goal.target_pose.pose.position.x
        self.y_target = goal.goal.target_pose.pose.position.y
        self.x_robot = []
        self.y_robot = []

    def GoalStatusCallback(self, msg: PlanningActionResult):
        self.state = msg.status.status
        if self.state == 2:
            self.y[0] += 1
        elif self.state == 3:
            self.y[1] += 1
        
        # Update the bar chart
        for bar, count in zip(self.reached_bar, self.y):
            bar.set_height(count)

    def update_plot(self, frame):
        self.ax[1].collections.clear()  # Clear any previous scatter plot
        self.robot_ln.set_data(self.x_robot, self.y_robot)
        self.target_ln.set_data(self.x_target, self.y_target)
        self.ax[0].set_xlim(-0.5, len(self.x) - 0.5)
        return self.robot_ln, self.target_ln, self.reached_bar

vis = Visualiser()

sub_odom = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
target_sub = rospy.Subscriber('/reaching_goal/goal', PlanningActionGoal, vis.TargetPoseCallback)
subscriber = rospy.Subscriber('/reaching_goal/result', PlanningActionResult, vis.GoalStatusCallback)
display(hbox)

ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit=True, interval=200)
plt.show(block=True)


### Distance to the Closest Obstacle

#### Widgets for displaying the distance and angle of the closest obstacle

In [ ]:
fixed_text = widgets.Label(value='The nearest obstacle is on:')
distance = widgets.FloatText(value="0.0",disabled=True,description='Distance')
angle = widgets.FloatText(value="0.0",disabled=True,description='Angle')
container = widgets.VBox([fixed_text,distance,angle])
container.layout = Layout(margin='auto auto auto auto',width='100%')

#### Extracting and calculating the distance and angle from topic /scam

In [ ]:
def odom_callback(msg:LaserScan):
    minimum_val = min(msg.ranges)
    index = msg.ranges.index(minimum_val)
    angle_val = msg.angle_min + (msg.angle_increment * index)
    distance.value = round(minimum_val,2)
    angle.value = round(angle_val*(180/pi),2)

f = rospy.Subscriber('/scan', LaserScan, odom_callback)



#### Displaying 

In [ ]:
display(container)

#### To stop the values from being unstable ( extract one value)

In [ ]:
f.unregister()